#랜덤 포레스트 
앙상블 학습의 대표적 주자 안정적인 성능으로 보편적으로 쓰임
앞에서 한 결정트리를 랜덤하게 만들어 숲을 만든 후 각 트리의 예측을 사용해 최종 예측을 만듬


#트리의 앙상블 학습
특성 총 개수의 제곱근 갯수만큼 무작위로 뽑아서 학습시킨다.

하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

중복이 허용된 샘플인 부트스트랩 샘플을 사용한다.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

전 수업과 같이 와인 데이터를 사용하고 훈련 세트와 테스트 세트로 나누기



In [2]:
wine = pd.read_csv('http://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input , test_input , train_target, test_target = train_test_split(data , target ,test_size = 0.2, random_state=42)

In [4]:
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True , n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞수업의 결정트리의 중요도 ['알코올 도수::'0.1245626, '당도:'0.86862934, 'pH:'0.0079144]

랜덤 포레스트의 중요도는 각 결정 트리에 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련 때문에 앞 수업과 다른 중요도를 보임
그 결과 하나의 특성에 과도하게 집중하지 않아 더많은 특성들이 훈련에 기여 함으로써 과대적합을 줄이고 일반화 성능을 높임

랜덤 포레스트의 검증세트의 역할을 하는 기능이 있는데 앞에서 말한 부트스트랩에 포합 되지 않는 나머지 샘플(OOB)를 이용해서 부트스트랩 샘플로 훈련한 결정트리를 평가 할수있다 

In [6]:
rf = RandomForestClassifier(oob_score=True , n_jobs=-1, random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


#엑스트라 트리 
기본적으로 랜덤 포레스트와 같지만 부트스트랩 샘플을 사용하지 않는다는 점이 있다 대신 노드 분할시 최적의 분할이 아닌 무작위로 분할한다

무작위로 분할하여 성능은 낮아지지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검중세트의 점수를 높힌다는 장점이 있다 

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

In [8]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


엑스트라 트리가 무작위성이 커서 더많은 결정트리를 훈련해야 하지만 랜덤하게 


노드를 분할하기 때문에 빠른 계산속도가 장점이다

In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


#그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법

경사 하강법을 사용하여 트리를 앙상블에 추가합니다. 분류엣는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용한다.

그레디언트 부스팅은 결정 트리를 계속 추가하면서 손실 함수의 가장 낮은 곳을 찾아 이동한다(학습률 매개변수로 속도를 조절하는 것 또한 같다).

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

In [16]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input,train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [17]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state=42)
scores =cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [19]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


#히스토그램 기반 그레이디언트 부스팅

입출력 특성을 256개의 구간으로 나눈다. 노드를 분할할 때 최적의 분할을 매우 빠르게 구할 수 있다.
입력에 누락된 특성이 있더라도 따로 전처리 할 필요가 없다.
사이킷런에서는 이 기능을 아직 테스트과정에 있다

In [22]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [23]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [24]:
# hgb에서의 특성 중요도 확인
from sklearn.inspection import permutation_importance

In [26]:
hgb.fit(train_input, train_target)
result= permutation_importance(hgb, train_input, train_target,
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [27]:
result= permutation_importance(hgb, test_input, test_target,
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [28]:
#HistGradientBoostingClassifier를 이용해 최종적인 성능 확인 
hgb.score(test_input, test_target)

0.8723076923076923

그레이디언트 부스팅 알고리즘을 구현한 라이브러리(XGBoost,LightGBM)

In [31]:
#XGBoost를 사용해서 와인 데이터 교차 검증 점수 확인
from xgboost import XGBClassifier

In [34]:
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [36]:
#LightGBM을 사용해서 와인 교차 검증 점수 확인 
from lightgbm import LGBMClassifier

In [37]:
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
